In [ ]:
!pip install langdetect

In [125]:
import pandas as pd
import nltk
from utils_processor.processor import Processor
import logging
import tensorflow as tf
from keras import layers, models
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from gensim.models import Word2Vec
import numpy as np
from keras import Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score
from typing import Tuple, List, Any
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
from typing import Tuple, Any

In [126]:
import os
import re

# Directorio donde están los archivos .txt
data_dir = 'data/books/'

# Lista para almacenar los textos y autores
texts = []
authors = []

# Leer todos los archivos .txt del directorio
for filename in os.listdir(data_dir):
    if filename.endswith('.txt'):

        with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
            text = f.read()
            texts.append(text)  # Almacenar el texto
            print(f"Procesando archivo: {filename}, lineas: ", len(text))
            
            # Buscar el nombre del autor
            author_match = re.search(r'Author:\s*(.+)', text)
            if author_match:
                author_name = author_match.group(1).strip()
                authors.append(author_name.lower())  # Almacenar el autor
            else:
                authors.append("Autor no encontrado")  # En caso de no encontrarlo
            
            # Mostrar un fragmento del texto (opcional)
            #print(text[:2500])

# Mostrar los autores encontrados
print("Lista de autores en orden:")
for i, author in enumerate(authors):
    print(f"{i+1}. {author}")

# Ahora tienes dos listas: 'texts' con los textos y 'authors' con los autores en el mismo orden
authors

Procesando archivo: Aeschylus.txt, lineas:  441833
Procesando archivo: Anonymous.txt, lineas:  388883
Procesando archivo: Apuleius.txt, lineas:  572020
Procesando archivo: Armando_Palacio_Valdes.txt, lineas:  318177
Procesando archivo: Dante_Alighieri.txt, lineas:  711986
Procesando archivo: Eca_de_Queiros.txt, lineas:  448001
Procesando archivo: Edgar_Allan_Poe.txt, lineas:  89819
Procesando archivo: Guillermo_Prieto.txt, lineas:  900466
Procesando archivo: Jose_Antonio.txt, lineas:  219288
Procesando archivo: Jose_Maria_Salaverria.txt, lineas:  205912
Procesando archivo: Miguel_Cane.txt, lineas:  643040
Procesando archivo: Miguel_de_Cervantes.txt, lineas:  356590
Procesando archivo: Miguel_de_Unamuno.txt, lineas:  339152
Procesando archivo: Ovid.txt, lineas:  158456
Procesando archivo: Pedro_Antonio_de_Alarcon.txt, lineas:  361771
Procesando archivo: Pedro_de_Angelis.txt, lineas:  583418
Procesando archivo: Pio_Baroja.txt, lineas:  394249
Procesando archivo: Roque_Barcia.txt, lineas:

['aeschylus',
 'anonymous',
 'apuleius',
 'armando palacio valdés',
 'dante alighieri',
 'eça de queirós',
 'edgar allan poe',
 'guillermo prieto',
 'ovid',
 'josé maría salaverría',
 'miguel cané',
 'miguel de cervantes saavedra',
 'miguel de unamuno',
 'ovid',
 'pedro antonio de alarcón',
 'martín del barco centenera',
 'pío baroja',
 'roque barcia',
 't. h. pardo de tavera',
 'vicente blasco ibáñez']

Verificar longitud de palabras por cada descripcion

In [127]:
df = pd.read_csv("data/classification/chistes_y_memes_clasificados_completos.csv")

In [128]:
def len_sentences(word):
    lista = word.split()
    return len(lista)

In [129]:
df["len"] = df.shape[0] * [0]
df["len"] = df["Chistes"].apply(lambda x: len_sentences(x))

In [130]:
df_len_chiste = df[df["len"]!=1]

In [131]:
df_len_chiste["len"].mean()

26.756023716412262

Aproximadamente son 27 palabras por chiste, teniendo esto en cuenta vamos a extraer los valores para poder empezar a sacar el dataset de datos negativos no chistosos. Nosotros colocaremos 30 palabras para que alcancen a formar por lo menos una oracion completa. Primero vamos a recortar los textos desde la porcion 8 en adelante, con la intencion de evitar el prologo y demas partes que estan en ingles de la libreria, asegurando que siempre vayan a ser datos en espanol y no en ingles ni nada por el estilo.

In [ ]:
def truncate_texts_from_7(texts):
    """
    Toma una lista de textos y reemplaza cada texto con las palabras desde la mitad hacia el final.
    
    :param texts: lista de textos (strings).
    :return: lista modificada con textos truncados desde la mitad.
    """
    truncated_texts = []
    
    for i in range(len(texts)):
        words = texts[i].split()  
        mid = len(words) // 7    
        truncated_texts.append(" ".join(words[mid:])) 
    
    return truncated_texts

In [133]:
for i in range(0,len(texts)):
    print(len(texts[i].split()))

77610
67910
100563
54157
124186
75643
15050
150274
37662
34003
107067
61519
60030
26951
56343
93185
66030
132441
10295
134028


In [ ]:
texts = truncate_texts_from_7(texts)

Para un dataset balanceado necesitamos 10k datos de negativos, entre los 20 libros, son 500 filas por cada una, eso quiere decir cada fila debe tener aproximadamente 30 palabras.

In [135]:
for i in range(0,len(texts)):
    print(len(texts[i].split()))

66523
58209
86197
46421
106446
64837
12900
128807
32282
29146
91772
52731
51455
23101
48294
79873
56598
113521
8825
114882


In [ ]:
import pandas as pd
import re

def split_into_sentences(text):
    """
    Divide un texto en oraciones completas utilizando signos de puntuación como delimitadores.
    """
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip()]

def generate_fragments(texts, words_per_fragment=30, fragments_per_book=500):
    """
    Genera fragmentos de tamaño específico (30 palabras) por cada libro (500 fragmentos por libro).
    
    :param texts: Lista de textos, cada uno correspondiente a un libro.
    :param words_per_fragment: Número de palabras por fragmento.
    :param fragments_per_book: Número de fragmentos a extraer por libro.
    :return: DataFrame con los fragmentos generados.
    """
    data = []

    for book_idx, book_text in enumerate(texts):
        sentences = split_into_sentences(book_text)
        all_words = []

        for sentence in sentences:
            all_words.extend(sentence.split())

        fragments = []
        for i in range(0, len(all_words), words_per_fragment):
            fragment = all_words[i:i + words_per_fragment]
            if len(fragment) == words_per_fragment: 
                fragments.append(" ".join(fragment))
            if len(fragments) == fragments_per_book:
                break  

        for fragment in fragments:
            data.append({"book_id": book_idx + 1, "text": fragment})

    return pd.DataFrame(data)



In [137]:
df_fragments = generate_fragments(texts, words_per_fragment=30, fragments_per_book=500)

In [138]:
df_fragments


,book_id,text
0,1,oigo en derredor de los muros el estruendoso r...
1,1,"ah! El aire brama enfurecido, azotado por las ..."
2,1,"¡Ay, ay! -- Una granizada de piedras viene sob..."
3,1,¡De Zeus venga el piadoso término rematador de...
4,1,¡Oh deidades prepotentes; excelsos dioses y di...
...,...,...
9719,20,este hombre de aspecto pacífico que acababa de...
9720,20,"y sus entusiasmos. Al mismo tiempo, tal vez de..."
9721,20,dirigiéndose á su primo--; tú has nacido en Ar...
9722,20,"de protesta, como si acabase de oir algo insul..."


In [139]:
from ydata_profiling import ProfileReport


In [121]:
ProfileReport(df_fragments, title="Profiling Report")

Render HTML: 100%|██████████| 1/1 [00:00<00:00, 10.15it/s]


Podemos observar que en algunas columnas no se logran completar las 500 oraciones, tal es el caso de Edgar Allan Poe, y esto es completamente normal porque es un libro de cortos poemas o cuentos. Sin embargo, esto no afecta los resutlados puesto que igual tenemos 9724 datos de control negativo que funcionan muy bien para evitar el humor.

In [ ]:
df_fragments.describe()

,book_id
count,9724.000000
mean,10.345125
std,5.705640
min,1.000000
25%,5.000000
50%,10.000000
75%,15.000000
max,20.000000


Ahora debemos estar completamente seguros que no se agrego nada en ingles en el dataset, esto lo hacemos con langdetect

In [145]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def detect_english_percentage_in_text(text):
    """
    Detecta el porcentaje de palabras en inglés en un texto.
    :param text: Texto para analizar.
    :return: Porcentaje de palabras en inglés.
    """
    words = text.split()
    total_words = len(words)
    if total_words == 0:
        return 0  # Retornar 0 si el texto está vacío

    english_count = 0

    for word in words:
        try:
            lang = detect(word)  # Detectar el idioma de cada palabra
            if lang == 'en':  # Si está en inglés, contar
                english_count += 1
        except LangDetectException:
            pass  # Ignorar palabras que no se pueden detectar

    english_percentage = (english_count / total_words) * 100
    return english_percentage


In [146]:
df_fragments['english_percentage'] = df_fragments['text'].apply(detect_english_percentage_in_text)

# Filtrar textos con más del 10% de inglés
result = df_fragments[df_fragments['english_percentage'] > 10]

# Mostrar resultados
print(result)

      book_id                                               text  \
26          1  de hombres se alimenta Ares. CORO ¡Oigo el rel...   
28          1  en Thebas. CORO ¡Oh consejo altísimo de los di...   
170         1  ETEOCLES Podéis decir cosas que sean de hacer,...   
199         1  fin con la infeliz raza. ¡Cosas para ser celeb...   
541         2  homenaje, siempre que el rey me lo permita.» Y...   
...       ...                                                ...   
9220       19  library of electronic works that could be free...   
9221       19  Gutenberg™ eBooks are often created from sever...   
9222       19  do not necessarily keep eBooks in compliance w...   
9223       19  about Project Gutenberg™, including how to mak...   
9575       20  gentes se interrogaban en los salones con el t...   

      english_percentage  
26             13.333333  
28             13.333333  
170            13.333333  
199            13.333333  
541            13.333333  
...                  

procedemos a eliminar estos datos con ids

In [160]:
df_fragments = df_fragments[~df_fragments.text.isin(result.text)]

In [178]:
df_fragments.reset_index(inplace=True)

In [180]:
df_fragments.drop(columns=["index"],inplace=True)

C:\Users\acost\AppData\Local\Temp\ipykernel_27912\4249385171.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fragments.drop(columns=["index"],inplace=True)


Una vez revisado este proceso estamos seguros que el dataset no contiene oraciones con mas de 10% de ingles y podemos guardarlo como datos negativos. Agregando label como 0 donde 0 representa no tiene humor y 1 es si tiene humor

In [181]:
df_fragments['label'] = df.shape[0] * 0 

C:\Users\acost\AppData\Local\Temp\ipykernel_27912\548559782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fragments['label'] = df.shape[0] * 0


In [183]:
df_fragments.drop(columns=["level_0"],inplace=True)

C:\Users\acost\AppData\Local\Temp\ipykernel_27912\3621380800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fragments.drop(columns=["level_0"],inplace=True)


In [185]:
df_fragments.drop(columns=["english_percentage","book_id"],inplace=True)
df_fragments

C:\Users\acost\AppData\Local\Temp\ipykernel_27912\1391298437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fragments.drop(columns=["english_percentage","book_id"],inplace=True)


,text,label
0,oigo en derredor de los muros el estruendoso r...,0
1,"ah! El aire brama enfurecido, azotado por las ...",0
2,"¡Ay, ay! -- Una granizada de piedras viene sob...",0
3,¡De Zeus venga el piadoso término rematador de...,0
4,¡Oh deidades prepotentes; excelsos dioses y di...,0
...,...,...
9479,este hombre de aspecto pacífico que acababa de...,0
9480,"y sus entusiasmos. Al mismo tiempo, tal vez de...",0
9481,dirigiéndose á su primo--; tú has nacido en Ar...,0
9482,"de protesta, como si acabase de oir algo insul...",0


In [186]:
df_fragments.to_csv("data/classification/data_with_no_humour.csv",index=False)